In [2]:
#허깅페이스 exaone << 강사님이 추천
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification

/home/user1/miniconda3/envs/sk_dnn1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
bert_model = AutoModel.from_pretrained("bert-base-uncased")
bert_tokenizer =AutoTokenizer.from_pretrained('bert-base-uncased')

In [4]:
model_id = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenized = tokenizer("독산에서 가장 맛있는 식당은?")
tokenized

{'input_ids': [0, 849, 2066, 27135, 3676, 4550, 2259, 5499, 2073, 35, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.convert_ids_to_tokens( tokenized["input_ids"])

['[CLS]', '독', '##산', '##에서', '가장', '맛있', '##는', '식당', '##은', '?', '[SEP]']

In [7]:
tokenizer.decode(tokenized["input_ids"])

'[CLS] 독산에서 가장 맛있는 식당은? [SEP]'

In [8]:
tokenizer(['독산에서 가장 맛있는 식당', '가산디지털단지에서 가장 맛있는 식당'])

{'input_ids': [[0, 849, 2066, 27135, 3676, 4550, 2259, 5499, 2], [0, 10485, 2220, 2118, 2669, 6694, 27135, 3676, 4550, 2259, 5499, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
bert_tokenizer(['독산에서 가장 맛있는 식당', '가산디지털단지에서 가장 맛있는 식당'])

{'input_ids': [[101, 1457, 30011, 30020, 29997, 30006, 30021, 29999, 30009, 29997, 30008, 1455, 30006, 30000, 30006, 30025, 100, 1461, 30019, 30020, 29993, 30006, 30025, 102], [101, 1455, 30006, 29997, 30006, 30021, 29993, 30019, 30000, 30019, 30003, 30008, 30022, 29993, 30006, 30021, 30000, 30019, 29999, 30009, 29997, 30008, 1455, 30006, 30000, 30006, 30025, 100, 1461, 30019, 30020, 29993, 30006, 30025, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [10]:
from datasets import load_dataset
klue_mrc_dataset = load_dataset('klue', 'mrc')

klue_tc_train = klue_mrc_dataset = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = klue_mrc_dataset = load_dataset('klue', 'ynat', split='validation')

klue_tc_train.features['label']

ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

In [11]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])

klue_tc_label = klue_tc_train.features['label']
klue_tc_train.features['label'].int2str(6)

'정치'

In [12]:
def make_str_label(batch):
    batch['label_str'] = klue_tc_label.int2str(batch['label'])
    return batch 

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)

test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

In [13]:
import numpy as np 
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
num_labels = len(klue_tc_train.features['label'].names)
model_id = "klue/roberta-base"
num_labels = len(klue_tc_train.features['label'].names)
model = AutoModelForSequenceClassification.from_pretrained(model_id, 
                    num_labels= num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def tokenize_function(example):
    return tokenizer(example['title'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2886.88 examples/s]


In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    push_to_hub=False
)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_55184/3459764511.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.668800
1000,0.543000


TrainOutput(global_step=1250, training_loss=0.5768625366210938, metrics={'train_runtime': 636.135, 'train_samples_per_second': 15.72, 'train_steps_per_second': 1.965, 'total_flos': 2631228672000000.0, 'train_loss': 0.5768625366210938, 'epoch': 1.0})